In [ ]:
!pip install pandas PyPDF2 openpyxl langchain chromadb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import PyPDF2
import os
from PyPDF2 import PdfReader

In [ ]:

# Import necessary libraries
import os
import pandas as pd
import PyPDF2
from PyPDF2 import PdfReader
from google.colab import files

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Function to extract data from Excel
def extract_data_from_excel(excel_path):
    data = pd.read_excel(excel_path)
    return data.to_string()  # Convert dataframe to string

# Upload files
uploaded = files.upload()

# Check uploaded files
print("Uploaded files:", uploaded.keys())

# Use the uploaded file names
pdf_file_name = "UNIT.pdf"
excel_file_name = "NMIT.xlsx"

# Check file existence
if os.path.isfile(pdf_file_name):
    print("PDF file exists:", pdf_file_name)
else:
    print("PDF file does not exist:", pdf_file_name)

if os.path.isfile(excel_file_name):
    print("Excel file exists:", excel_file_name)
else:
    print("Excel file does not exist:", excel_file_name)

# Example usage
try:
    pdf_text = extract_text_from_pdf(pdf_file_name)
    excel_text = extract_data_from_excel(excel_file_name)
    print("PDF Text:\n", pdf_text)
    print("Excel Text:\n", excel_text)
except FileNotFoundError as e:
    print(e)


Saving NMIT.xlsx to NMIT.xlsx
Saving UNIT.pdf to UNIT.pdf
Uploaded files: dict_keys(['NMIT.xlsx', 'UNIT.pdf'])
PDF file exists: UNIT.pdf
Excel file exists: NMIT.xlsx
PDF Text:
 Lecture Notes: AUTOMOTIVE ELECTRONICS  (18ECE823)                        
 
Dr. Vinaykumar R, Assistant Professor, ECE, NMIT, Bengaluru  1 
 UNIT – 5 
Automotive Control Systems and Diagnostic Systems:  
Active Safety Systems:  ABS, TCS, ESP, Brake assist etc.  
Passive Safety Systems:  Airbag systems, Advanced Driver Assistance Systems (ADAS): 
Combining computer vision techniques as pattern recognition, feature extraction, learning, 
tracking, 3D vision, etc. to develop real -time algorithms able to assist the driving activity. 
Examples o f assistance applications: Lane Departure Warning, Collision Warning, Automatic 
Cruise Control, Headlights Control, Connected Cars technology and trends towards 
Autonomous vehicles.  
Functional Safety:  Need for safety systems, safety concept, and safety process for produ

In [ ]:
!pip install chromadb openai


In [ ]:
!pip install chromadb sentence-transformers langchain

import os
import pandas as pd

from google.colab import files
import chromadb
from sentence_transformers import SentenceTransformer

from langchain.text_splitter import CharacterTextSplitter


# Initialize ChromaDB client
client = chromadb.Client()

# Delete the existing collection if it exists
try:
    client.delete_collection(name="documents")
except Exception as e:
    print(f"Error deleting collection: {e}")

# Create or get an existing collection
collection = client.create_collection(name="documents")

# Initialize SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings using SentenceTransformers
def get_embeddings(text):
    return model.encode(text).tolist()  # Convert to list to store in ChromaDB

# Prepare data for indexing
documents = [
    {"id": "pdf_1", "text": pdf_text, "metadata": {"type": "pdf"}},
    {"id": "excel_1", "text": excel_text, "metadata": {"type": "excel"}}
]

# Index documents
for doc in documents:
    embeddings = get_embeddings(doc["text"])
    collection.add(doc["id"], [embeddings], doc["metadata"])  # Wrap embeddings in a list

# Query the vector database to find similar documents
def query_similar_documents(query_text):
    query_embedding = get_embeddings(query_text)
    results = collection.query([query_embedding])  # Pass embeddings as a list of lists
    return results

# Update the embedding of a specific document
def update_document_embedding(doc_id, new_text):
    new_embedding = get_embeddings(new_text)
    collection.update(doc_id, [new_embedding])  # Wrap embeddings in a list
    print(f"Updated document {doc_id}")

# Delete a document from the index
def delete_document(doc_id):
    collection.delete(doc_id)
    print(f"Deleted document {doc_id}")

# Replace the embedding of one document with another
def replace_document_embedding(doc_id, new_text):
    new_embedding = get_embeddings(new_text)
    collection.update(doc_id, [new_embedding])  # Wrap embeddings in a list
    print(f"Replaced document {doc_id}")

# Example usage of the operations
query_text = "Example query text for similarity search"
results = query_similar_documents(query_text)
print("Query results:", results)

# Update a document
new_pdf_text = "New content for PDF"
update_document_embedding("pdf_1", new_pdf_text)

# Delete a document
delete_document("excel_1")

# Replace a document embedding
replace_document_embedding("pdf_1", "Completely new content for PDF")

# Function to generate a summary using LangChain
!pip install langchain openai
!pip install langchain transformers
!pip install transformers



from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import pipeline



Query results: {'ids': [['excel_1', 'pdf_1']], 'distances': [[1.7689683437347412, 1.9758055210113525]], 'metadatas': [[{'type': 'excel'}, {'type': 'pdf'}]], 'embeddings': None, 'documents': [[None, None]], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
Updated document pdf_1
Deleted document excel_1
Replaced document pdf_1


In [ ]:

# Initialize the summarization pipeline with a specific model
model_name = "facebook/bart-large-cnn"
summarizer = pipeline("summarization", model=model_name)

def generate_summary(text):
    chunk_size = 512
    summaries = []

    num_chunks = (len(text) + chunk_size - 1) // chunk_size
    print(f"Processing {num_chunks} chunks...")

    for i in range(0, len(text), chunk_size):
        chunk = text[i:i + chunk_size]
        print(f"Processing chunk {i // chunk_size + 1}/{num_chunks}...")
        summary = summarizer(chunk, max_length=150, min_length=30, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    return " ".join(summaries)

pdf_text = extract_text_from_pdf(pdf_file_name)

try:
    summary = generate_summary(pdf_text)
    print("Summary of the PDF document:\n", summary)
except KeyboardInterrupt:
    print("Process interrupted by user.")


Processing 94 chunks...
Processing chunk 1/94...


Your max_length is set to 150, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)


Processing chunk 2/94...


Your max_length is set to 150, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


Processing chunk 3/94...


...

Processing chunk 91/94...
Processing chunk 92/94...


Your max_length is set to 150, but your input_length is only 148. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)


Processing chunk 93/94...


Your max_length is set to 150, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


Processing chunk 94/94...
Summary of the PDF document:
 Automotive Control Systems and Diagnostic Systems. Active Safety Systems: ABS, TCS, ESP, Brake assist etc. Airbag systems, Advanced Driver Assistance Systems (ADAS) Autonomous vehicles. Lane Departure Warning, Collision Warning, Automatic Cruise Control, Headlights Control, Connected Cars technology and trends towards autonomous vehicles. Antilock Breaking System (ABS) is used in advanced automobiles to prevent slip and locking of wheel after brakes applied. In normal  braking situation the driver control the brakes, but during severs braking or on slippery  roadways when driver the wheels to  approach loc. The ABS modulates the brake line  pressure independent of the pedal force. It allowed  driver to control the car easier, even on roads  with low traffic. The brain of antilock braking  system consist Electronic Control Unit (ECU),  wheel speed sensor  and hydraulic modulator. ABS is a closed circuit, hence it used the  feedback

In [ ]:
excel_text = extract_data_from_excel(excel_file_name)

In [ ]:
summary_excel = generate_summary(excel_text)

Processing 14 chunks...
Processing chunk 1/14...
Processing chunk 2/14...
Processing chunk 3/14...
Processing chunk 4/14...
Processing chunk 5/14...
Processing chunk 6/14...
Processing chunk 7/14...
Processing chunk 8/14...
Processing chunk 9/14...
Processing chunk 10/14...
Processing chunk 11/14...
Processing chunk 12/14...
Processing chunk 13/14...


Your max_length is set to 150, but your input_length is only 116. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)


Processing chunk 14/14...


In [ ]:
print("Summary of the Excel document:\n", summary_excel)

Summary of the Excel document:
 Nitte Meenakshi Institute of Technology - YEHLANKA                1    20064607   192871  Hack Diva 2024  7430627. Ragini Nagaraja   9902681552  Registration - Pending  Female. Nitte Meenakshi Institute of Technology - YEHLANKA2   20073761    192871  Hack Diva 2024  7429639  Registration - Pending  Female   Moksha, Paramesh, Ravikumar, Varsha, Vinod. Nitte Meenakshi Institute of Technology - YEHLANKA. Registration - Pending  Female. Sirisha, Jayam and Nagatulasi.  Registration - Pending  Female  Nitte Meenakshi Institute of Technology - YEHLANKA                7.    192871  Hack Diva 2024. Sneha Kudarihalmath, Kavya Manjunath, K Shreya Krishna Murthy. Pending  Female  Nitte Meenakshi Institute of Technology - YEHLANKA. shreyabujji2002@gmail.com  8431825250  Registration - Pending  female. Nitte Meenakshi Institute of Technology - YEHLANKA.11   20234753    192871  Hack Diva 2024. Pratheeksha, Vijay, Nayakbh. and Prathinaik. Registration - Pending  Female.

1. File Upload and Extraction:
Uploads Files: Allows users to upload PDF and Excel files.
Text Extraction from PDF: Extracts text from the uploaded PDF using PyPDF2.
Data Extraction from Excel: Extracts data from the uploaded Excel file using pandas.
2. ChromaDB Integration:
ChromaDB Client Initialization: Initializes a ChromaDB client for vector database operations.
Delete Collection: Deletes any existing "documents" collection if it exists.
Create Collection: Creates a new collection named "documents" to store document embeddings.
3. Text Embedding Generation:
Model Initialization: Initializes a SentenceTransformer model (all-MiniLM-L6-v2) to generate text embeddings.
Generate Embeddings: Converts extracted text (from both PDF and Excel) into embeddings using the SentenceTransformer model.
Index Documents: Adds the generated embeddings and metadata (type of document) to the ChromaDB collection.
4. Vector Database Operations:
Query Similar Documents: Accepts a query, generates embeddings, and searches for similar documents in the vector database.
Update Document Embedding: Updates the embedding of a specific document in the collection.
Delete Document: Deletes a specific document from the collection.
Replace Document Embedding: Replaces the embedding of one document with new content.
5. Summarization (Hugging Face Transformers):
Hugging Face Pipeline Initialization: Initializes a summarization pipeline using a Hugging Face pre-trained model (facebook/bart-large-cnn).
Generate Summaries:
Generates a summary for the extracted text from the PDF file.
Generates a summary for the extracted data from the Excel file.